In [ ]:
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.externals import joblib
from numpy.linalg import norm

class PathFinder: 
    def __init__(self,model_source,video_source=0):
        self.clf = joblib.load(model_source) 
        self.cap=cv2.VideoCapture(video_source)
            
    def __hog(self,img,step):
        gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
        gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
        mag, ang = cv2.cartToPolar(gx, gy)
        bin_n = 16 # Number of bins
        bin = np.int32(bin_n*ang/(2*np.pi))
        bin_cells = []
        mag_cells = []
        cellx = celly = step   
        for i in range(0,int(img.shape[0]/celly)):
            for j in range(0,int(img.shape[1]/cellx)):
                bin_cells.append(bin[i*celly : i*celly+celly, j*cellx : j*cellx+cellx])
                mag_cells.append(mag[i*celly : i*celly+celly, j*cellx : j*cellx+cellx])   
        hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
        hist = np.hstack(hists)
        eps = 1e-7
        hist /= hist.sum() + eps
        hist = np.sqrt(hist)
        hist /= norm(hist) + eps
        return hist
    
    def StartRide(self):
        frame=self.cap.read()[1]
        while(self.cap.isOpened()):
            frame=self.cap.read()[1]
            frame = cv2.GaussianBlur(frame,(3,3),0)
            predict=self.__detectSigns(frame)
            #HERE ADD YOUR MODULES
            cv2.imshow('frame2',frame)
            if cv2.waitKey(2) & 0xFF == ord("q"):
                break
            
        self.cap.release()
        cv2.destroyAllWindows()
    
    def __detectSigns(self,frame):
        frame = cv2.resize(frame,(320, 240), interpolation = cv2.INTER_CUBIC)
        r,g,b = cv2.split(frame)    
        circles = cv2.HoughCircles(b,cv2.HOUGH_GRADIENT,1,50,param1=56,param2=80,minRadius=20,maxRadius=100) #let's change params
        pred=[]
        if circles is not None:
            circles = np.round(circles[0, :]).astype("int")
            for (x, y, r) in circles:
                cut_frame = frame[y-r:y+r,x-r:x+r]
                cut_frame = cv2.resize(cut_frame,(20, 20), interpolation = cv2.INTER_CUBIC)
                hog=self.__hog(cut_frame,8)
                pred.append(self.clf.predict([hog])) #change to clf.predict_proba              
        return pred
    
    
    

In [ ]:
One = PathFinder('model.pkl',"test.mp4")
One.StartRide()